<a href="https://colab.research.google.com/github/MarcosVCS/Projeto-Final---Engenharia-de-Dados---SoulCode/blob/main/COEXP_pyspark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#1 - instalar e importar

In [ ]:
import pandas as pd

In [ ]:
#Configurações de numero maximo de colunas em exibição
pd.set_option('display.max_columns',30)

In [ ]:
import numpy as np

google

In [ ]:
#instalar o gc file system
!pip install gcsfs

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
#importar o cloud storage
from google.cloud import storage
import os

In [ ]:
# FAZER O ACESSO A BUCKET
serviceAccount = '/content/chave_bucket_JSON.json' # Chave de acesso à Bucket no Cloud Storage
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = serviceAccount

client = storage.Client()             

# indicar a bucket
bucket = client.get_bucket('bc26-datasets-comex')

In [ ]:
# # Para acessar o arquivo que está na bucket
# bucket.blob('bc26-datasets-comex/EXP_COMPLETA (NCM).csv')
# #para enviar
# blob = bucket.blob('preco-combustiveis-2013-2022')
# blob.upload_from_filename('/content/preco-combustiveis-2013-2022')

In [ ]:
from google.cloud.storage.blob import Blob 
import requests

In [ ]:
from google.oauth2 import service_account 
import os
import pandas_gbq 

from IPython.display import clear_output

##spark

In [ ]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 4.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 KB 14.8 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845512 sha256=fd100f46989c3c9463656e2d62a28123aad95a7587bf474da30f97e2e16205ad
  Stored in directory: /root/.cache/pip/wheels/43/dc/11/ec201cd671da62fa9c5cc77078235e40722170ceba231d7598
Successfully built pyspark


In [ ]:
#importa o pysaprk
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql.types import *

In [ ]:
from pyspark.sql.window import Window

In [ ]:
spark = (
    SparkSession.builder
    .master('local')
    .appName('data-frames')
    .config('spark.ui.port','4050')
    .config('spark.jars', 'https://storage.googleapis.com/hadoop-lib/gcs/gcs-connector-hadoop2-latest.jar')
    .getOrCreate()
)

In [ ]:
spark

#2 - caminho das bases

In [ ]:
#Caminho das base no bucket - versão 1
# #tratada
# path_im = 'gs://bc26-datasets-comex/comex-tratado/IMP_2012-2022_tratada (MUN).parquet'
# path_ip = 'gs://bc26-datasets-comex/comex-tratado/IMP_2012-2022_tratada (NCM).parquet'
# #original
# path_em = 'gs://bc26-datasets-comex/EXP_COMPLETA (MUN).csv'
# path_ep = 'gs://bc26-datasets-comex/EXP_COMPLETA (NCM).csv'
# #tradato - checar o link
# path_comb = 'gs://bc26-datasets-comex/preco-combustiveis-2013-2022-TRATADO'

# #base de categorias
# path_cat = 'gs://bc26-datasets-comex/comex-tratado/imp-exp-classificacoes'

In [ ]:
#versão arrumada por Erika
#Caminho das base no bucket
#tratada
path_im = 'gs://bc26-datasets-comex/comex-tratado/IMP_2012-2022_tratada (MUN).parquet'
path_ip = 'gs://bc26-datasets-comex/comex-tratado/IMP_2012-2022_tratada (NCM).parquet'
#tratado - dar erro
path_em = 'gs://bc26-datasets-comex/comex-tratado/EXP_2012_2022_tratado(MUN)'
path_ep = 'gs://bc26-datasets-comex/comex-tratado/EXP_2012_2022_tratado(NCM)'
#tratado - checar o link
path_comb = 'gs://bc26-datasets-comex/preco-combustiveis-2013-2022-TRATADO'

#base de categorias
path_cat = 'gs://bc26-datasets-comex/comex-tratado/imp-exp-classificacoes'

In [ ]:
#Caminho das base no bucket - minha nova versão

#tratada importação
path_imp_mun = 'gs://bc26-datasets-comex/comex-tratado/IMP_2012-2022_tratada (MUN).parquet'
path_imp_prod = 'gs://bc26-datasets-comex/comex-tratado/IMP_2012-2022_tratada (NCM).parquet'

#tratada exportação 
path_exp_mun = 'gs://bc26-datasets-comex/comex-tratado/IMP_2012-2022_tratada (MUN).parquet'

path_exp_prod = 'gs://bc26-datasets-comex/comex-tratado/EXP_2013-2022_tratado(NCM).csv'

#tratada combustíveis
path_comb = 'gs://bc26-datasets-comex/preco-combustiveis-2013-2022-TRATAMENTO-OK'
#tratasa combustívies com volume
path_comb_vol = 'gs://bc26-datasets-comex/combustivel-tratado/volume_preco_combustiveis_mensal_2013_2022' 


In [ ]:
# que base é essa - é uma versão de exportação-municipio
# path_comex = 'gs://bc26-datasets-comex/bc26-datasets-comex/comex-tratado'

In [ ]:
#base de categorias
path_cat = 'gs://bc26-datasets-comex/comex-tratado/imp-exp-classificacoes'

#base de auxilio
path_pais = 'gs://bc26-datasets-comex/comex-tratado/nome-pais'
path_bloco = 'gs://bc26-datasets-comex/comex-tratado/bloco-economico'
path_ecom = 'gs://bc26-datasets-comex/comex-tratado/bloco-geografico'

# 3 - base

##3a - importação municipio

In [ ]:
#criar um dataframe pelo pyspark
dfimp_mun = ( spark.read.format('parquet')
                 .option('header', 'true')
                 .option('inferschema', 'true')
                 .option('delimiter', ',')
                 .load(path_imp_mun))

In [ ]:
#caso o comando falhe, carregar a base por pandas e tranformar
#criar dataframe com as importações por município
# df_im = pd.read_parquet(path_imp_mun)
#olar a base
# df_im.head(2)

In [ ]:
#transforma para spark
# dfimp_mun = spark.createDataFrame(df_im)

In [ ]:
#ver a base
dfimp_mun.show()

+--------+----+--------+------+-------------+------------+-----------+-------------------------------+------------+----+
|     mes| SH4|cod_pais|estado|cod_municipio|peso_liquido|valor_dolar|descricoes__sistema_harmonizado|   municipio| ano|
+--------+----+--------+------+-------------+------------+-----------+-------------------------------+------------+----+
|    maio|4016|     160|    RS|      4314902|         826|       4341|           Compostos heteroc...|Porto Alegre|2021|
|dezembro|8477|     245|    RS|      4314902|           1|        512|           Fatos de saia-cas...|Porto Alegre|2021|
| outubro|7326|     245|    RS|      4314902|       16152|     112832|           Outros papéis, ca...|Porto Alegre|2021|
|   março|8708|     160|    RS|      4314902|      199184|     762402|           Chapéus e outros ...|Porto Alegre|2021|
|novembro|8542|     493|    RS|      4314902|           1|       2864|           Lenços de assoar ...|Porto Alegre|2021|
|novembro|7013|     160|    RS| 

In [ ]:
#se usou a transformação
# del df_im

In [ ]:
#ver o esquema
dfimp_mun.printSchema()

root
 |-- mes: string (nullable = true)
 |-- SH4: long (nullable = true)
 |-- cod_pais: long (nullable = true)
 |-- estado: string (nullable = true)
 |-- cod_municipio: long (nullable = true)
 |-- peso_liquido: long (nullable = true)
 |-- valor_dolar: long (nullable = true)
 |-- descricoes__sistema_harmonizado: string (nullable = true)
 |-- municipio: string (nullable = true)
 |-- ano: integer (nullable = true)



In [ ]:
dfimp_mun.describe().show()
#desisti depois de 30 min

In [ ]:
#resumir os dados - comando demorado!
dfimp_mun.select(F.col('ano'), F.col('mes'), F.col('valor_dolar'), F.col('peso_liquido')).summary().show()

In [ ]:
#criar windons sobre os estado ordenados pelo valor e pelo volume
w1 = Window.partitionBy(F.col('estado')).orderBy(F.col('valor_dolar').desc())
w2 = Window.partitionBy(F.col('estado')).orderBy(F.col('peso_liquido').desc())

In [ ]:
#rankear os produtos dentro de cada estado pelo valor
dfimp_mun.withColumn('rank', F.rank().over(w1)).show()

#cada produto é importado de um mesmo pais?

+---------+----+--------+------+-------------+------------+-----------+-------------------------------+--------------------+----+----+
|      mes| SH4|cod_pais|estado|cod_municipio|peso_liquido|valor_dolar|descricoes__sistema_harmonizado|           municipio| ano|rank|
+---------+----+--------+------+-------------+------------+-----------+-------------------------------+--------------------+----+----+
| novembro|3102|     556|    SC|      4216206|   235755435|  102546437|           Ácidos monocarbox...|São Francisco do Sul|2021|   1|
|     maio|3102|     556|    SC|      4216206|   110155938|   96113699|           Ácidos monocarbox...|São Francisco do Sul|2022|   2|
|  outubro|8703|     493|    SC|      4209102|     9021499|   84679167|           Chapéus e outros ...|           Joinville|2018|   3|
|fevereiro|3102|     556|    SC|      4216206|   156521310|   80203394|           Ácidos monocarbox...|São Francisco do Sul|2022|   4|
|    julho|8703|      63|    SC|      4209102|     4574

In [ ]:
#rankear os produtos dentro de cada estado pelo volume
dfimp_mun.withColumn('rank', F.rank().over(w2)).show()

+---------+----+--------+------+-------------+------------+-----------+-------------------------------+--------------------+----+----+
|      mes| SH4|cod_pais|estado|cod_municipio|peso_liquido|valor_dolar|descricoes__sistema_harmonizado|           municipio| ano|rank|
+---------+----+--------+------+-------------+------------+-----------+-------------------------------+--------------------+----+----+
| novembro|3102|     556|    SC|      4216206|   235755435|  102546437|           Ácidos monocarbox...|São Francisco do Sul|2021|   1|
|     maio|2713|     249|    SC|      4208203|   200589931|   39185289|           Derivados sulfona...|              Itajaí|2022|   2|
|    julho|2713|     249|    SC|      4208203|   193573465|   38781592|           Derivados sulfona...|              Itajaí|2022|   3|
|fevereiro|3102|     556|    SC|      4216206|   156521310|   80203394|           Ácidos monocarbox...|São Francisco do Sul|2022|   4|
| novembro|2713|     249|    SC|      4208203|   137068

In [ ]:
#criar windons sobre os paises ordenados pelo valor e pelo volume
w3 = Window.partitionBy(F.col('cod_pais')).orderBy(F.col('valor_dolar').desc())
w4 = Window.partitionBy(F.col('cod_pais')).orderBy(F.col('peso_liquido').desc())

In [ ]:
#rankear os produtos dentro de cada pais pelo valor
dfimp_mun.withColumn('rank', F.rank().over(w3)).show()

+--------+----+--------+------+-------------+------------+-----------+-------------------------------+----------------+----+----+
|     mes| SH4|cod_pais|estado|cod_municipio|peso_liquido|valor_dolar|descricoes__sistema_harmonizado|       municipio| ano|rank|
+--------+----+--------+------+-------------+------------+-----------+-------------------------------+----------------+----+----+
|   julho|8703|       0|    PR|      4106902|        2812|      22294|           Chapéus e outros ...|        Curitiba|2014|   1|
|  agosto|9401|       0|    RS|      4314902|        2560|       7083|           Reservatórios, to...|    Porto Alegre|2012|   2|
|   junho|8537|       0|    SP|      3440606|           8|       1021|           Soutiens, cintas,...|     Porto Feliz|2014|   3|
|   março|2710|      65|    PE|      2607208|    40327043|   42241190|           Derivados sulfona...|         Ipojuca|2012|   1|
| janeiro|2710|      65|    MA|      2111300|    39379796|   36473309|           Derivados

In [ ]:
#rankear os produtos dentro de cada pais pelo volume
dfimp_mun.withColumn('rank', F.rank().over(w4)).show()

+--------+----+--------+------+-------------+------------+-----------+-------------------------------+----------------+----+----+
|     mes| SH4|cod_pais|estado|cod_municipio|peso_liquido|valor_dolar|descricoes__sistema_harmonizado|       municipio| ano|rank|
+--------+----+--------+------+-------------+------------+-----------+-------------------------------+----------------+----+----+
|   julho|8703|       0|    PR|      4106902|        2812|      22294|           Chapéus e outros ...|        Curitiba|2014|   1|
|  agosto|9401|       0|    RS|      4314902|        2560|       7083|           Reservatórios, to...|    Porto Alegre|2012|   2|
|   junho|8537|       0|    SP|      3440606|           8|       1021|           Soutiens, cintas,...|     Porto Feliz|2014|   3|
|   março|2710|      65|    PE|      2607208|    40327043|   42241190|           Derivados sulfona...|         Ipojuca|2012|   1|
| janeiro|2710|      65|    MA|      2111300|    39379796|   36473309|           Derivados

Fazer esses rankes em um tabela nova agrupada com pelo produto

In [ ]:
#deletar a base por espaço
# del dfimp_mun

## 3b - base importação produto

In [ ]:
dfimp_prod = ( spark.read.format('parquet')
                 .option('header', 'true')
                 .option('inferschema', 'true')
                 .option('delimiter', ',')
                 .load(path_imp_prod))

In [ ]:
#caso o comando falhe, carregar a base por pandas e tranformar
#criar dataframe com as importações por município
# df_ip = pd.read_parquet(path_imp_prod)
#olhar a base
# df_ip.head(2)

In [ ]:
#transforma para spark
# dfimp_prod = spark.createDataFrame(df_ip)

In [ ]:
#ver a base
dfimp_prod.show()

+--------+----------+-----------+--------+------+-------+---------------+------------+-----------+--------------------+----------------+---------------+----+
|     mes|mercadoria|cod_unidade|cod_pais|estado|cod_urf|qnt_estatistica|peso_liquido|valor_dolar|             produto|  unidade_medida|meio_transporte| ano|
+--------+----------+-----------+--------+------+-------+---------------+------------+-----------+--------------------+----------------+---------------+----+
|novembro|  73269090|         10|     493|    SP| 817700|           1409|        1409|      10031|Outras obras de f...|NUMERO (UNIDADE)|          AEREA|2021|
|   março|  73269090|         10|     249|    RJ| 817700|           2859|        2859|     459689|Outras obras de f...|NUMERO (UNIDADE)|          AEREA|2021|
|dezembro|  73269090|         10|     160|    RJ| 717700|            315|         315|      49945|Outras obras de f...|NUMERO (UNIDADE)|          AEREA|2021|
|dezembro|  73269090|         10|     275|    SP| 81

In [ ]:
#se usou a transformação
# del df_ip

In [ ]:
#ver o esquema
dfimp_prod.printSchema()

root
 |-- mes: string (nullable = true)
 |-- mercadoria: long (nullable = true)
 |-- cod_unidade: long (nullable = true)
 |-- cod_pais: long (nullable = true)
 |-- estado: string (nullable = true)
 |-- cod_urf: long (nullable = true)
 |-- qnt_estatistica: long (nullable = true)
 |-- peso_liquido: long (nullable = true)
 |-- valor_dolar: long (nullable = true)
 |-- produto: string (nullable = true)
 |-- unidade_medida: string (nullable = true)
 |-- meio_transporte: string (nullable = true)
 |-- ano: integer (nullable = true)



In [ ]:
dfimp_prod.describe().show()

In [ ]:
#resumir os dados - comando demorado!
dfimp_prod.select(F.col('valor_dolar'), F.col('peso_liquido')).summary().show()

In [ ]:
#rankear os produtos dentro de cada estado pelo valor
dfimp_prod.withColumn('rank', F.rank().over(w1)).show()

+---------+----------+-----------+--------+------+-------+---------------+------------+-----------+--------------------+----------------+---------------+----+----+
|      mes|mercadoria|cod_unidade|cod_pais|estado|cod_urf|qnt_estatistica|peso_liquido|valor_dolar|             produto|  unidade_medida|meio_transporte| ano|rank|
+---------+----------+-----------+--------+------+-------+---------------+------------+-----------+--------------------+----------------+---------------+----+----+
|    abril|  74031100|         21|     158|    SC| 927700|          13644|    13643839|  125574572|Cátodos e seus el...|NUMERO (UNIDADE)|       MARITIMA|2021|   1|
|  outubro|  74031100|         21|     158|    SC| 927700|          13264|    13261941|  125239077|Cátodos e seus el...|NUMERO (UNIDADE)|       MARITIMA|2021|   2|
|   agosto|  74031100|         21|     158|    SC| 927700|          13043|    13040993|  124482248|Cátodos e seus el...|NUMERO (UNIDADE)|       MARITIMA|2021|   3|
|    março|  740

In [ ]:
#rankear os produtos dentro de cada estado pelo volume
dfimp_prod.withColumn('rank', F.rank().over(w2)).show()

+---------+----------+-----------+--------+------+-------+---------------+------------+-----------+--------------------+----------------+---------------+----+----+
|      mes|mercadoria|cod_unidade|cod_pais|estado|cod_urf|qnt_estatistica|peso_liquido|valor_dolar|             produto|  unidade_medida|meio_transporte| ano|rank|
+---------+----------+-----------+--------+------+-------+---------------+------------+-----------+--------------------+----------------+---------------+----+----+
|     maio|  27131100|         10|     249|    SC| 927502|      210307698|   210307698|   40948522|Coque de petróleo...|NUMERO (UNIDADE)|       MARITIMA|2022|   1|
|    julho|  27131100|         10|     249|    SC| 927502|      193573465|   193573465|   38781592|Coque de petróleo...|NUMERO (UNIDADE)|       MARITIMA|2022|   2|
| novembro|  31021010|         10|     556|    SC| 927700|      163506605|   163506605|   69696391|Ureia, mesmo em s...|NUMERO (UNIDADE)|       MARITIMA|2021|   3|
|  outubro|  271

In [ ]:
#rankear os produtos dentro de cada pais pelo valor
dfimp_prod.withColumn('rank', F.rank().over(w3)).show()

+--------+----------+-----------+--------+------+-------+---------------+------------+-----------+--------------------+----------------+---------------+----+----+
|     mes|mercadoria|cod_unidade|cod_pais|estado|cod_urf|qnt_estatistica|peso_liquido|valor_dolar|             produto|  unidade_medida|meio_transporte| ano|rank|
+--------+----------+-----------+--------+------+-------+---------------+------------+-----------+--------------------+----------------+---------------+----+----+
|   julho|  87032410|         11|       0|    PR| 927700|              1|        2812|      22294|Automóveis com mo...|NUMERO (UNIDADE)|       MARITIMA|2014|   1|
|  agosto|  94017900|         11|       0|    RS|1017700|            512|        2560|       7083|Outros assentos c...|NUMERO (UNIDADE)|       MARITIMA|2012|   2|
|   junho|  85371090|         10|       0|    SP| 817700|              8|           8|       1021|Outros quadros, e...|NUMERO (UNIDADE)|          AEREA|2014|   3|
|   março|  27101911| 

In [ ]:
#rankear os produtos dentro de cada pais pelo volume
dfimp_prod.withColumn('rank', F.rank().over(w4)).show()

+--------+----------+-----------+--------+------+-------+---------------+------------+-----------+--------------------+----------------+---------------+----+----+
|     mes|mercadoria|cod_unidade|cod_pais|estado|cod_urf|qnt_estatistica|peso_liquido|valor_dolar|             produto|  unidade_medida|meio_transporte| ano|rank|
+--------+----------+-----------+--------+------+-------+---------------+------------+-----------+--------------------+----------------+---------------+----+----+
|   julho|  87032410|         11|       0|    PR| 927700|              1|        2812|      22294|Automóveis com mo...|NUMERO (UNIDADE)|       MARITIMA|2014|   1|
|  agosto|  94017900|         11|       0|    RS|1017700|            512|        2560|       7083|Outros assentos c...|NUMERO (UNIDADE)|       MARITIMA|2012|   2|
|   junho|  85371090|         10|       0|    SP| 817700|              8|           8|       1021|Outros quadros, e...|NUMERO (UNIDADE)|          AEREA|2014|   3|
|   março|  27101911| 

In [ ]:
#criar windons sobre os meios de transporte ordenados pelo valor e pelo volume
w5 = Window.partitionBy(F.col('meio_transporte')).orderBy(F.col('valor_dolar').desc())
w6 = Window.partitionBy(F.col('meio_transporte')).orderBy(F.col('peso_liquido').desc())

In [ ]:
#rankear os produtos dentro de cada meio de transporte pelo valor
dfimp_prod.withColumn('rank', F.rank().over(w5)).show()

+---------+----------+-----------+--------+------+-------+---------------+------------+-----------+--------------------+----------------+--------------------+----+----+
|      mes|mercadoria|cod_unidade|cod_pais|estado|cod_urf|qnt_estatistica|peso_liquido|valor_dolar|             produto|  unidade_medida|     meio_transporte| ano|rank|
+---------+----------+-----------+--------+------+-------+---------------+------------+-----------+--------------------+----------------+--------------------+----+----+
|    abril|  27112100|         10|      97|    MS| 145200|      733060583|   733060583|  333017602|Gás natural no es...|NUMERO (UNIDADE)|CONDUTO/REDE DE T...|2012|   1|
|  janeiro|  27112100|         10|      97|    MS| 145200|      747441654|   747441654|  330730259|Gás natural no es...|NUMERO (UNIDADE)|CONDUTO/REDE DE T...|2014|   2|
| novembro|  27112100|         10|      97|    MS| 145200|      745762906|   745762906|  330685879|Gás natural no es...|NUMERO (UNIDADE)|CONDUTO/REDE DE T.

In [ ]:
#rankear os produtos dentro de cada meio de transporte pelo volume
dfimp_prod.withColumn('rank', F.rank().over(w6)).show()

+---------+----------+-----------+--------+------+-------+---------------+------------+-----------+--------------------+----------------+--------------------+----+----+
|      mes|mercadoria|cod_unidade|cod_pais|estado|cod_urf|qnt_estatistica|peso_liquido|valor_dolar|             produto|  unidade_medida|     meio_transporte| ano|rank|
+---------+----------+-----------+--------+------+-------+---------------+------------+-----------+--------------------+----------------+--------------------+----+----+
|fevereiro|  27112100|         10|      97|    MS| 145200|      748755022|   748755022|  318572299|Gás natural no es...|NUMERO (UNIDADE)|CONDUTO/REDE DE T...|2014|   1|
|  janeiro|  27112100|         10|      97|    MS| 145200|      747441654|   747441654|  330730259|Gás natural no es...|NUMERO (UNIDADE)|CONDUTO/REDE DE T...|2014|   2|
| novembro|  27112100|         10|      97|    MS| 145200|      745762906|   745762906|  330685879|Gás natural no es...|NUMERO (UNIDADE)|CONDUTO/REDE DE T.

In [ ]:
#deletar a base por espaço
# del dfimp_prod

## 3c - base exportação municipio

In [ ]:
dfexp_mun = ( spark.read.format('parquet')
                 .option('header', 'true')
                 .option('inferschema', 'true')
                 .option('delimiter', ',')
                 .load(path_exp_mun))

In [ ]:
#caso o comando falhe, carregar a base por pandas e tranformar
#criar dataframe com as importações por município
# df_em = pd.read_parquet(path_exp_mun)
#olar a base
# df_em.head(2)

In [ ]:
#transforma para spark
# dfexp_mun = spark.createDataFrame(df_em)

In [ ]:
#ver a base
dfexp_mun.show()

+--------+----+--------+------+-------------+------------+-----------+-------------------------------+------------+----+
|     mes| SH4|cod_pais|estado|cod_municipio|peso_liquido|valor_dolar|descricoes__sistema_harmonizado|   municipio| ano|
+--------+----+--------+------+-------------+------------+-----------+-------------------------------+------------+----+
|    maio|4016|     160|    RS|      4314902|         826|       4341|           Compostos heteroc...|Porto Alegre|2021|
|dezembro|8477|     245|    RS|      4314902|           1|        512|           Fatos de saia-cas...|Porto Alegre|2021|
| outubro|7326|     245|    RS|      4314902|       16152|     112832|           Outros papéis, ca...|Porto Alegre|2021|
|   março|8708|     160|    RS|      4314902|      199184|     762402|           Chapéus e outros ...|Porto Alegre|2021|
|novembro|8542|     493|    RS|      4314902|           1|       2864|           Lenços de assoar ...|Porto Alegre|2021|
|novembro|7013|     160|    RS| 

In [ ]:
#se usou a transformação
# del df_em

In [ ]:
#ver o esquema
dfexp_mun.printSchema()

root
 |-- mes: string (nullable = true)
 |-- SH4: long (nullable = true)
 |-- cod_pais: long (nullable = true)
 |-- estado: string (nullable = true)
 |-- cod_municipio: long (nullable = true)
 |-- peso_liquido: long (nullable = true)
 |-- valor_dolar: long (nullable = true)
 |-- descricoes__sistema_harmonizado: string (nullable = true)
 |-- municipio: string (nullable = true)
 |-- ano: integer (nullable = true)



In [ ]:
dfexp_mun.describe().show()

In [ ]:
#resumir os dados - comando demorado!
dfexp_mun.select(F.col('ano'), F.col('mes'), F.col('valor_dolar'), F.col('peso_liquido')).summary().show()

In [ ]:
#rankear os produtos dentro de cada estado pelo valor
dfexp_mun.withColumn('rank', F.rank().over(w1)).show()

+---------+----+--------+------+-------------+------------+-----------+-------------------------------+--------------------+----+----+
|      mes| SH4|cod_pais|estado|cod_municipio|peso_liquido|valor_dolar|descricoes__sistema_harmonizado|           municipio| ano|rank|
+---------+----+--------+------+-------------+------------+-----------+-------------------------------+--------------------+----+----+
| novembro|3102|     556|    SC|      4216206|   235755435|  102546437|           Ácidos monocarbox...|São Francisco do Sul|2021|   1|
|     maio|3102|     556|    SC|      4216206|   110155938|   96113699|           Ácidos monocarbox...|São Francisco do Sul|2022|   2|
|  outubro|8703|     493|    SC|      4209102|     9021499|   84679167|           Chapéus e outros ...|           Joinville|2018|   3|
|fevereiro|3102|     556|    SC|      4216206|   156521310|   80203394|           Ácidos monocarbox...|São Francisco do Sul|2022|   4|
|    julho|8703|      63|    SC|      4209102|     4574

In [ ]:
#rankear os produtos dentro de cada estado pelo volume
dfexp_mun.withColumn('rank', F.rank().over(w2)).show()

+---------+----+--------+------+-------------+------------+-----------+-------------------------------+--------------------+----+----+
|      mes| SH4|cod_pais|estado|cod_municipio|peso_liquido|valor_dolar|descricoes__sistema_harmonizado|           municipio| ano|rank|
+---------+----+--------+------+-------------+------------+-----------+-------------------------------+--------------------+----+----+
| novembro|3102|     556|    SC|      4216206|   235755435|  102546437|           Ácidos monocarbox...|São Francisco do Sul|2021|   1|
|     maio|2713|     249|    SC|      4208203|   200589931|   39185289|           Derivados sulfona...|              Itajaí|2022|   2|
|    julho|2713|     249|    SC|      4208203|   193573465|   38781592|           Derivados sulfona...|              Itajaí|2022|   3|
|fevereiro|3102|     556|    SC|      4216206|   156521310|   80203394|           Ácidos monocarbox...|São Francisco do Sul|2022|   4|
| novembro|2713|     249|    SC|      4208203|   137068

In [ ]:
#rankear os produtos dentro de cada pais pelo valor
dfexp_mun.withColumn('rank', F.rank().over(w3)).show()

+--------+----+--------+------+-------------+------------+-----------+-------------------------------+----------------+----+----+
|     mes| SH4|cod_pais|estado|cod_municipio|peso_liquido|valor_dolar|descricoes__sistema_harmonizado|       municipio| ano|rank|
+--------+----+--------+------+-------------+------------+-----------+-------------------------------+----------------+----+----+
|   julho|8703|       0|    PR|      4106902|        2812|      22294|           Chapéus e outros ...|        Curitiba|2014|   1|
|  agosto|9401|       0|    RS|      4314902|        2560|       7083|           Reservatórios, to...|    Porto Alegre|2012|   2|
|   junho|8537|       0|    SP|      3440606|           8|       1021|           Soutiens, cintas,...|     Porto Feliz|2014|   3|
|   março|2710|      65|    PE|      2607208|    40327043|   42241190|           Derivados sulfona...|         Ipojuca|2012|   1|
| janeiro|2710|      65|    MA|      2111300|    39379796|   36473309|           Derivados

In [ ]:
#rankear os produtos dentro de cada pais pelo volume
dfexp_mun.withColumn('rank', F.rank().over(w4)).show()

+--------+----+--------+------+-------------+------------+-----------+-------------------------------+----------------+----+----+
|     mes| SH4|cod_pais|estado|cod_municipio|peso_liquido|valor_dolar|descricoes__sistema_harmonizado|       municipio| ano|rank|
+--------+----+--------+------+-------------+------------+-----------+-------------------------------+----------------+----+----+
|   julho|8703|       0|    PR|      4106902|        2812|      22294|           Chapéus e outros ...|        Curitiba|2014|   1|
|  agosto|9401|       0|    RS|      4314902|        2560|       7083|           Reservatórios, to...|    Porto Alegre|2012|   2|
|   junho|8537|       0|    SP|      3440606|           8|       1021|           Soutiens, cintas,...|     Porto Feliz|2014|   3|
|   março|2710|      65|    PE|      2607208|    40327043|   42241190|           Derivados sulfona...|         Ipojuca|2012|   1|
| janeiro|2710|      65|    MA|      2111300|    39379796|   36473309|           Derivados

In [ ]:
#deletar a base por espaço
# del dfexp_mun

## 3d - base exportação produto

In [ ]:
dfexp_prod = ( spark.read.format('csv')
                 .option('header', 'true')
                 .option('inferschema', 'true')
                 .option('delimiter', ',')
                 .load(path_exp_prod))

In [ ]:
#caso o comando falhe, carregar a base por pandas e tranformar
#criar dataframe com as importações por município
# df_ep = pd.read_parquet(path_exp_prod)
#olar a base
# df_ep.head(2)

In [ ]:
#transforma para spark
# dfexp_prod = spark.createDataFrame(df_ep)

In [ ]:
#ver a base
dfexp_prod.show()

+----+---------+--------------+------------------+--------+------+---------------+-------+---------------+------------+-----------+-----------------+--------------------+
| ano|      mes|cod_mercadoria|           unidade|cod_pais|estado|meio_transporte|cod_urf|qnt_estatistica|peso_liquido|valor_dolar|  urf_localizacao|          mercadoria|
+----+---------+--------------+------------------+--------+------+---------------+-------+---------------+------------+-----------+-----------------+--------------------+
|2013|    Março|      30067000|Quilograma líquido|     169|    SP|       Marítima| 817800|           1447|        1447|      27538|  PORTO DE SANTOS|Preparações em ge...|
|2013|  Janeiro|      30067000|Quilograma líquido|     493|    SP|       Marítima| 817800|           1512|        1512|      26435|  PORTO DE SANTOS|Preparações em ge...|
|2013|    Julho|      30067000|Quilograma líquido|     580|    SP|       Marítima| 817800|           1526|        1526|      19997|  PORTO DE SAN

In [ ]:
#se usou a transformação
# del df_ep

In [ ]:
#ver o esquema
dfexp_prod.printSchema()

root
 |-- ano: string (nullable = true)
 |-- mes: string (nullable = true)
 |-- cod_mercadoria: integer (nullable = true)
 |-- unidade: string (nullable = true)
 |-- cod_pais: integer (nullable = true)
 |-- estado: string (nullable = true)
 |-- meio_transporte: string (nullable = true)
 |-- cod_urf: integer (nullable = true)
 |-- qnt_estatistica: long (nullable = true)
 |-- peso_liquido: long (nullable = true)
 |-- valor_dolar: integer (nullable = true)
 |-- urf_localizacao: string (nullable = true)
 |-- mercadoria: string (nullable = true)



In [ ]:
dfexp_prod.describe().show()

In [ ]:
#resumir os dados - comando demorado!
dfexp_prod.select(F.col('valor_dolar'), F.col('peso_liquido')).summary().show()

In [ ]:
#rankear os produtos dentro de cada estado pelo valor
dfexp_prod.withColumn('rank', F.rank().over(w1)).show()

+----+---------+--------------+------------------+--------+------+---------------+-------+---------------+------------+-----------+---------------+--------------------+----+
| ano|      mes|cod_mercadoria|           unidade|cod_pais|estado|meio_transporte|cod_urf|qnt_estatistica|peso_liquido|valor_dolar|urf_localizacao|          mercadoria|rank|
+----+---------+--------------+------------------+--------+------+---------------+-------+---------------+------------+-----------+---------------+--------------------+----+
|2013|  Outubro|      21069010|Quilograma líquido|     850|    AM|     Rodoviária| 260151|         724243|      724243|   30605549|      PACARAIMA|Outras preparaçõe...|   1|
|2014|    Junho|      21069010|Quilograma líquido|     850|    AM|     Rodoviária| 260151|         653191|      653191|   28932566|      PACARAIMA|Outras preparaçõe...|   2|
|2014|    Abril|      21069010|Quilograma líquido|     850|    AM|     Rodoviária| 260151|         633819|      633819|   27299954

In [ ]:
#rankear os produtos dentro de cada estado pelo volume
dfexp_prod.withColumn('rank', F.rank().over(w2)).show()

+----+---------+--------------+--------------------+--------+------+---------------+-------+---------------+------------+-----------+-----------------+--------------------+----+
| ano|      mes|cod_mercadoria|             unidade|cod_pais|estado|meio_transporte|cod_urf|qnt_estatistica|peso_liquido|valor_dolar|  urf_localizacao|          mercadoria|rank|
+----+---------+--------------+--------------------+--------+------+---------------+-------+---------------+------------+-----------+-----------------+--------------------+----+
|2016|Fevereiro|      12019000|Tonelada métrica ...|     160|    AM|       Marítima| 227600|          29714|    29714231|   10257947|  PORTO DE MANAUS|Soja, mesmo tritu...|   1|
|2019|  Janeiro|      12019000|Tonelada métrica ...|     160|    AM|       Marítima| 227600|          21435|    21434816|    7677952|  PORTO DE MANAUS|Soja, mesmo tritu...|   2|
|2019|Fevereiro|      12019000|Tonelada métrica ...|     160|    AM|       Marítima| 227600|          15004|  

In [ ]:
#rankear os produtos dentro de cada pais pelo valor
dfexp_prod.withColumn('rank', F.rank().over(w3)).show()

+----+---------+--------------+------------------+--------+------+---------------+-------+---------------+------------+-----------+--------------------+--------------------+----+
| ano|      mes|cod_mercadoria|           unidade|cod_pais|estado|meio_transporte|cod_urf|qnt_estatistica|peso_liquido|valor_dolar|     urf_localizacao|          mercadoria|rank|
+----+---------+--------------+------------------+--------+------+---------------+-------+---------------+------------+-----------+--------------------+--------------------+----+
|2018|    Julho|      10063011|Quilograma líquido|      31|    RS|       Marítima|1017700|       14945360|    14945360|    5739018|  PORTO DE RIO GR...|Arroz semibranque...|   1|
|2013| Dezembro|      93062100|Quilograma líquido|      31|    PR|       Marítima| 817800|          25114|       25114|    2419845|     PORTO DE SANTOS|Cartuchos para es...|   2|
|2017|    Março|      84291190|    Número unidade|      31|    SP|       Marítima| 817800|              8

In [ ]:
#rankear os produtos dentro de cada pais pelo volume
dfexp_prod.withColumn('rank', F.rank().over(w4)).show()

+----+---------+--------------+------------------+--------+------+---------------+-------+---------------+------------+-----------+--------------------+--------------------+----+
| ano|      mes|cod_mercadoria|           unidade|cod_pais|estado|meio_transporte|cod_urf|qnt_estatistica|peso_liquido|valor_dolar|     urf_localizacao|          mercadoria|rank|
+----+---------+--------------+------------------+--------+------+---------------+-------+---------------+------------+-----------+--------------------+--------------------+----+
|2018|    Julho|      10063011|Quilograma líquido|      31|    RS|       Marítima|1017700|       14945360|    14945360|    5739018|  PORTO DE RIO GR...|Arroz semibranque...|   1|
|2019|    Junho|      27101922|      Metro cúbico|      31|    RJ|       Marítima| 717800|           3959|     3763693|    1614624|    PORTO DE ITAGUAI|            Fuel oil|   2|
|2020|  Janeiro|      27101922|      Metro cúbico|      31|    RJ|       Marítima| 717600|           1815

In [ ]:
#rankear os produtos dentro de cada meio de transporte pelo valor
dfexp_prod.withColumn('rank', F.rank().over(w5)).show()

+----+---------+--------------+--------------------+--------+------+---------------+-------+---------------+------------+-----------+--------------------+--------------------+----+
| ano|      mes|cod_mercadoria|             unidade|cod_pais|estado|meio_transporte|cod_urf|qnt_estatistica|peso_liquido|valor_dolar|     urf_localizacao|          mercadoria|rank|
+----+---------+--------------+--------------------+--------+------+---------------+-------+---------------+------------+-----------+--------------------+--------------------+----+
|2018| Dezembro|      27090010|        Metro cúbico|     160|    RJ|        Fluvial| 717800|         594994|   528002394|  282362699|    PORTO DE ITAGUAI|Óleos brutos de p...|   1|
|2014|    Junho|      26011100|  Quilograma líquido|      63|    MS|        Fluvial| 145200|     1274397000|  1274397000|  103797419|             CORUMBA|Minérios de ferro...|   2|
|2014|     Maio|      26011100|  Quilograma líquido|      63|    MS|        Fluvial| 145200|   

In [ ]:
#rankear os produtos dentro de cada meio de transporte pelo volume
dfexp_prod.withColumn('rank', F.rank().over(w6)).show()

+----+---------+--------------+------------------+--------+------+---------------+-------+---------------+------------+-----------+------------------+--------------------+----+
| ano|      mes|cod_mercadoria|           unidade|cod_pais|estado|meio_transporte|cod_urf|qnt_estatistica|peso_liquido|valor_dolar|   urf_localizacao|          mercadoria|rank|
+----+---------+--------------+------------------+--------+------+---------------+-------+---------------+------------+-----------+------------------+--------------------+----+
|2014|    Junho|      26011100|Quilograma líquido|      63|    MS|        Fluvial| 145200|     1274397000|  1274397000|  103797419|           CORUMBA|Minérios de ferro...|   1|
|2014|     Maio|      26011100|Quilograma líquido|      63|    MS|        Fluvial| 145200|     1200505000|  1200505000|  101166474|           CORUMBA|Minérios de ferro...|   2|
|2014|    Julho|      26011100|Quilograma líquido|      63|    MS|        Fluvial| 145200|      887094000|   887094

In [ ]:
#deletar a base por espaço
# del dfexp_prod

## 3e - base combustivel preço

In [ ]:
dfcomb = ( spark.read.format('parquet')
                 .option('header', 'true')
                 .option('inferschema', 'true')
                 .option('delimiter', ',')
                 .load(path_comb))

In [ ]:
#criar o dataframe com os preço dos combustivies
# df_cp = pd.read_parquet(path_comb_preco)
# df_cp.head(2)

In [ ]:
#transforma para spark
# dfcomb_prec= spark.createDataFrame(df_cp)

In [ ]:
#ver a base
dfcomb.show()

+------------+----------+------------+----------------+--------------------+----------------+----------------------+--------------+-------------------+---------------------+-----------------+-----------------+--------------------+---------------------+
|data_inicial|data_final|      regiao|          estado|           municipio|         produto|num_postos_pesquisados|unidade_medida|preco_medio_revenda|desvio_padrao_revenda|preco_min_revenda|preco_max_revenda|margem_media_revenda|coef_variacao_revenda|
+------------+----------+------------+----------------+--------------------+----------------+----------------------+--------------+-------------------+---------------------+-----------------+-----------------+--------------------+---------------------+
|  2012-12-30|2013-01-05|CENTRO OESTE|DISTRITO FEDERAL|            BRASILIA|ETANOL HIDRATADO|                    80|          R$/l|              2.273|                0.023|             2.19|            2.299|               0.405|           

In [ ]:
#se usou a transformação
# del df_cp

In [ ]:
#ver o esquema
dfcomb.printSchema()

root
 |-- data_inicial: date (nullable = true)
 |-- data_final: date (nullable = true)
 |-- regiao: string (nullable = true)
 |-- estado: string (nullable = true)
 |-- municipio: string (nullable = true)
 |-- produto: string (nullable = true)
 |-- num_postos_pesquisados: integer (nullable = true)
 |-- unidade_medida: string (nullable = true)
 |-- preco_medio_revenda: float (nullable = true)
 |-- desvio_padrao_revenda: float (nullable = true)
 |-- preco_min_revenda: float (nullable = true)
 |-- preco_max_revenda: float (nullable = true)
 |-- margem_media_revenda: float (nullable = true)
 |-- coef_variacao_revenda: float (nullable = true)



In [ ]:
dfcomb.describe().show()

+-------+------------+---------+----------+----------------+----------------------+--------------+-------------------+---------------------+------------------+------------------+--------------------+---------------------+
|summary|      regiao|   estado| municipio|         produto|num_postos_pesquisados|unidade_medida|preco_medio_revenda|desvio_padrao_revenda| preco_min_revenda| preco_max_revenda|margem_media_revenda|coef_variacao_revenda|
+-------+------------+---------+----------+----------------+----------------------+--------------+-------------------+---------------------+------------------+------------------+--------------------+---------------------+
|  count|     1190920|  1190920|   1190920|         1190920|               1190920|       1190920|            1190920|              1190920|           1190920|           1190920|             1190920|              1190920|
|   mean|        null|     null|      null|            null|    10.612314849024283|          null| 14.5719599747

In [ ]:
#resumir os dados - comando demorado!
dfcomb.summary().show()

+-------+------------+---------+----------+----------------+----------------------+--------------+-------------------+---------------------+------------------+------------------+--------------------+---------------------+
|summary|      regiao|   estado| municipio|         produto|num_postos_pesquisados|unidade_medida|preco_medio_revenda|desvio_padrao_revenda| preco_min_revenda| preco_max_revenda|margem_media_revenda|coef_variacao_revenda|
+-------+------------+---------+----------+----------------+----------------------+--------------+-------------------+---------------------+------------------+------------------+--------------------+---------------------+
|  count|     1190920|  1190920|   1190920|         1190920|               1190920|       1190920|            1190920|              1190920|           1190920|           1190920|             1190920|              1190920|
|   mean|        null|     null|      null|            null|    10.612314849024283|          null| 14.5719599747

In [ ]:
#criar windons por produto ordenada pela data_final
w7 = Window.partitionBy(F.col('produto')).orderBy(F.col('data_final').asc())

In [ ]:
#ver o registro anterior ordenado por data
dfcomb.withColumn('lag', F.lag('preco_medio_revenda', 1).over(w7)).show()
dfvar = dfcomb.withColumn('lag', F.lag('preco_medio_revenda', 1).over(w7))

+------------+----------+------------+----------------+--------------------+----------------+----------------------+--------------+-------------------+---------------------+-----------------+-----------------+--------------------+---------------------+-----+
|data_inicial|data_final|      regiao|          estado|           municipio|         produto|num_postos_pesquisados|unidade_medida|preco_medio_revenda|desvio_padrao_revenda|preco_min_revenda|preco_max_revenda|margem_media_revenda|coef_variacao_revenda|  lag|
+------------+----------+------------+----------------+--------------------+----------------+----------------------+--------------+-------------------+---------------------+-----------------+-----------------+--------------------+---------------------+-----+
|  2012-12-30|2013-01-05|CENTRO OESTE|DISTRITO FEDERAL|            BRASILIA|ETANOL HIDRATADO|                    80|          R$/l|              2.273|                0.023|             2.19|            2.299|              

In [ ]:
#criar coluna com o aumento percentual do preço de uma senama para outra
dfvar = dfvar.withColumn('var_percentual', (F.col('preco_medio_revenda') - F.col('lag'))*100/F.col('lag'))

In [ ]:
dfvar.show()

+------------+----------+------------+----------------+--------------------+----------------+----------------------+--------------+-------------------+---------------------+-----------------+-----------------+--------------------+---------------------+-----+-------------------+
|data_inicial|data_final|      regiao|          estado|           municipio|         produto|num_postos_pesquisados|unidade_medida|preco_medio_revenda|desvio_padrao_revenda|preco_min_revenda|preco_max_revenda|margem_media_revenda|coef_variacao_revenda|  lag|     var_percentual|
+------------+----------+------------+----------------+--------------------+----------------+----------------------+--------------+-------------------+---------------------+-----------------+-----------------+--------------------+---------------------+-----+-------------------+
|  2012-12-30|2013-01-05|CENTRO OESTE|DISTRITO FEDERAL|            BRASILIA|ETANOL HIDRATADO|                    80|          R$/l|              2.273|            

In [ ]:
#deletar a base por espaço
# del dfcomb

## 3f - base combustivel volume

In [ ]:
dfcomb_vol = ( spark.read.format('parquet')
                 .option('header', 'true')
                 .option('inferschema', 'true')
                 .option('delimiter', ',')
                 .load(path_comb_vol))

In [ ]:
#caso o comando falhe, carregar a base por pandas e tranformar
#criar dataframe com as importações por município
# df_cv = pd.read_parquet(path_comb_vol)
#olar a base
# df_cv.head(2)

In [ ]:
#transforma para spark
# dfcomb_vol = spark.createDataFrame(df_cv)

In [ ]:
#ver a base
dfcomb_vol.show()

+---------+-------------------+----------------+------------+----------------------+-------------------+---------------------+-----------------+-----------------+--------------------+---------------------+----+
|      mes|             estado|         produto| qtde_vendas|num_postos_pesquisados|preco_medio_revenda|desvio_padrao_revenda|preco_min_revenda|preco_max_revenda|margem_media_revenda|coef_variacao_revenda| ano|
+---------+-------------------+----------------+------------+----------------------+-------------------+---------------------+-----------------+-----------------+--------------------+---------------------+----+
|FEVEREIRO|           AMAZONAS|  GASOLINA COMUM| 4.4716714E7|                  14.0|                4.5|                0.048|             4.41|             4.54|               0.788|                0.011|2018|
| DEZEMBRO|              BAHIA|ETANOL HIDRATADO|  5.640484E7|                  11.0|               3.39|                0.109|             3.24|            

In [ ]:
#se usou a transformação
# del df_cp

In [ ]:
#ver o esquema
dfcomb_vol.printSchema()

root
 |-- mes: string (nullable = true)
 |-- estado: string (nullable = true)
 |-- produto: string (nullable = true)
 |-- qtde_vendas: double (nullable = true)
 |-- num_postos_pesquisados: double (nullable = true)
 |-- preco_medio_revenda: double (nullable = true)
 |-- desvio_padrao_revenda: double (nullable = true)
 |-- preco_min_revenda: double (nullable = true)
 |-- preco_max_revenda: double (nullable = true)
 |-- margem_media_revenda: double (nullable = true)
 |-- coef_variacao_revenda: double (nullable = true)
 |-- ano: integer (nullable = true)



In [ ]:
dfcomb_vol.describe().show()

+-------+--------+---------+----------------+--------------------+----------------------+-------------------+---------------------+------------------+------------------+--------------------+---------------------+------------------+
|summary|     mes|   estado|         produto|         qtde_vendas|num_postos_pesquisados|preco_medio_revenda|desvio_padrao_revenda| preco_min_revenda| preco_max_revenda|margem_media_revenda|coef_variacao_revenda|               ano|
+-------+--------+---------+----------------+--------------------+----------------------+-------------------+---------------------+------------------+------------------+--------------------+---------------------+------------------+
|  count|    9440|     9440|            9440|                9440|                  9440|               9440|                 9440|              9440|              9440|                7255|                 9440|              9440|
|   mean|    null|     null|            null| 6.899972620173848E7|     1

In [ ]:
#resumir os dados - comando demorado!
dfcomb_vol.summary().show()

+-------+--------+---------+----------------+--------------------+----------------------+-------------------+---------------------+------------------+------------------+--------------------+---------------------+------------------+
|summary|     mes|   estado|         produto|         qtde_vendas|num_postos_pesquisados|preco_medio_revenda|desvio_padrao_revenda| preco_min_revenda| preco_max_revenda|margem_media_revenda|coef_variacao_revenda|               ano|
+-------+--------+---------+----------------+--------------------+----------------------+-------------------+---------------------+------------------+------------------+--------------------+---------------------+------------------+
|  count|    9440|     9440|            9440|                9440|                  9440|               9440|                 9440|              9440|              9440|                7255|                 9440|              9440|
|   mean|    null|     null|            null| 6.899972620173848E7|     1

In [ ]:
#criar windons por produto ordenado pela data
#criar coluna com data!
#mes por extenso
w8 = Window.partitionBy(F.col('produto')).orderBy(F.col('ano').asc(),F.col('mes').asc())

In [ ]:
#ver o registro anterior ordenado por data
dfcomb_vol.withColumn('lag_preco', F.lag('preco_medio_revenda', 1).over(w8)).withColumn('lag_qtd', F.lag('preco_medio_revenda', 1).over(w7)).show()


In [ ]:
# #criar coluna com o aumento percentual do preço de uma senama para outra
dfvar_vol = dfvar.withColumn('var_percentual_preco', (F.col('preco_medio_revenda') - F.col('lag_preco'))/F.col('lag_preco'))
.withColumn('var_percentual_qtd', (F.col('preco_medio_revenda') - F.col('lag_qtd'))/F.col('lag_qtd'))

In [ ]:
#deletar a base por espaço
# del dfcomb_vol

##3h - base misteriosa

In [ ]:
#olhar base misteriosa
#é a base de exportaçã municpipio com defeito!

In [ ]:
# dfcomex = ( spark.read.format('csv')
                 .option('header', 'true')
                 .option('inferschema', 'true')
                 .option('delimiter', ',')
                 .load(path_comex))

In [ ]:
# dfcomex.show()

+-------+----+--------+-------------------+------+--------------------+--------------+--------------------+------------+-----------+
|    _c0| ano|     mes|          municipio|estado|                pais|ncm_mercadoria|          mercadoria|peso_liquido|valor_dolar|
+-------+----+--------+-------------------+------+--------------------+--------------+--------------------+------------+-----------+
|9651203|2013|Setembro|São José dos Campos|    SP|              Europa|          8503|Partes reconhecív...|          43|     8689.0|
|9651204|2013|Setembro|São José dos Campos|    SP| União Europeia - UE|          8503|Partes reconhecív...|          43|     8689.0|
|9651205|2016|   Abril|São José dos Campos|    SP|              Europa|          8503|Partes reconhecív...|         197|     4344.0|
|9651206|2016|   Abril|São José dos Campos|    SP| União Europeia - UE|          8503|Partes reconhecív...|         197|     4344.0|
|9651207|2016|    Maio|São José dos Campos|    SP|              Europ

In [ ]:
# dfcomex.dtypes

In [ ]:
# del dfcomex

In [ ]:
#se não abri, olhar em pandas
# df_comex = read_csv(path_comex)
# df_comex.head(3)

## 3g - base auxiliares

In [ ]:
# montar com pandas
dfpais = ( spark.read.format('csv')
                 .option('header', 'true')
                 .option('inferschema', 'true')
                 .option('delimiter', ',')
                 .load(path_pais))

# df_pais = pd.read_csv(path_pais)

dfbloco = ( spark.read.format('csv')
                 .option('header', 'true')
                 .option('inferschema', 'true')
                 .option('delimiter', ',')
                 .load(path_bloco))

# df_bloco = pd.read_csv(path_bloco)

df_ecom = ( spark.read.format('csv')
                 .option('header', 'true')
                 .option('inferschema', 'true')
                 .option('delimiter', ',')
                 .load(path_ecom))

# df_ecom = pd.read_csv(path_ecom)

In [ ]:
#olhar
# df_pais
#converter
# dfpais = spark.createDataFrame(df_pais)

In [ ]:
# dfpais.describe().show()

+-------+-----------------+----------+
|summary|          CO_PAIS|   NO_PAIS|
+-------+-----------------+----------+
|  count|              281|       281|
|   mean|449.4697508896797|      null|
| stddev| 266.870822101005|      null|
|    min|                0|A Designar|
|    max|              999|     Índia|
+-------+-----------------+----------+



#Filtro para ligar expotação/importação a combustíveis

In [ ]:
#lista com os cogido da produtos relacionados a base combustível
sh6 = [271000, 271011, 271012,271019,271111,271119,271121,271129,290511,382600]
ncm = [27100029, 27100041,27100042,27100049,27101159,27101259,27101921,27111100,27111910,27112100,27112990,29051100 ,38260000]
#o sh4 é inclui algumas variações
sh4 = [2710, 2711, 2905, 3826]

In [ ]:
#filtro na base de importação por município
dfimp_mun.filter(F.col('SH4').isin(sh4)).show()

+---------+----+--------+------+-------------+------------+-----------+-------------------------------+------------+----+
|      mes| SH4|cod_pais|estado|cod_municipio|peso_liquido|valor_dolar|descricoes__sistema_harmonizado|   municipio| ano|
+---------+----+--------+------+-------------+------------+-----------+-------------------------------+------------+----+
|    junho|2710|     361|    RS|      4314902|       39970|      39218|           Derivados sulfona...|Porto Alegre|2021|
| setembro|2710|     361|    RS|      4314902|       39890|      42015|           Derivados sulfona...|Porto Alegre|2021|
| novembro|2710|      23|    RS|      4314902|       25200|     114023|           Derivados sulfona...|Porto Alegre|2021|
|     maio|2710|     249|    RS|      4314902|        7254|      72043|           Derivados sulfona...|Porto Alegre|2021|
| setembro|2710|     275|    RS|      4314902|         800|       2564|           Derivados sulfona...|Porto Alegre|2021|
|    julho|2905|     160

In [ ]:
#filtro na base de exportação por município
dfexp_mun.filter(F.col('SH4').isin(sh4)).show()

+---------+----+--------+------+-------------+------------+-----------+-------------------------------+------------+----+
|      mes| SH4|cod_pais|estado|cod_municipio|peso_liquido|valor_dolar|descricoes__sistema_harmonizado|   municipio| ano|
+---------+----+--------+------+-------------+------------+-----------+-------------------------------+------------+----+
|    junho|2710|     361|    RS|      4314902|       39970|      39218|           Derivados sulfona...|Porto Alegre|2021|
| setembro|2710|     361|    RS|      4314902|       39890|      42015|           Derivados sulfona...|Porto Alegre|2021|
| novembro|2710|      23|    RS|      4314902|       25200|     114023|           Derivados sulfona...|Porto Alegre|2021|
|     maio|2710|     249|    RS|      4314902|        7254|      72043|           Derivados sulfona...|Porto Alegre|2021|
| setembro|2710|     275|    RS|      4314902|         800|       2564|           Derivados sulfona...|Porto Alegre|2021|
|    julho|2905|     160

In [ ]:
#filtro na base de importação por produto
dfimp_prod.filter(F.col('mercadoria').isin(ncm)).show()

+---------+----------+-----------+--------+------+-------+---------------+------------+-----------+--------------------+----------------+---------------+----+
|      mes|mercadoria|cod_unidade|cod_pais|estado|cod_urf|qnt_estatistica|peso_liquido|valor_dolar|             produto|  unidade_medida|meio_transporte| ano|
+---------+----------+-----------+--------+------+-------+---------------+------------+-----------+--------------------+----------------+---------------+----+
| dezembro|  29051100|         10|     190|    SP| 817700|              8|           8|        135|Metanol (álcool m...|NUMERO (UNIDADE)|          AEREA|2021|
|    junho|  29051100|         10|     249|    GO| 817700|              0|           0|       7973|Metanol (álcool m...|NUMERO (UNIDADE)|          AEREA|2021|
| setembro|  29051100|         10|     249|    GO| 817700|              0|           0|       2289|Metanol (álcool m...|NUMERO (UNIDADE)|          AEREA|2021|
|    março|  29051100|         10|     249|   

In [ ]:
#filtro na base de exportação por produto
dfexp_prod.filter(F.col('cod_mercadoria').isin(ncm)).show()

+----+--------+--------------+------------+--------+------+---------------+-------+---------------+------------+-----------+-----------------+--------------------+
| ano|     mes|cod_mercadoria|     unidade|cod_pais|estado|meio_transporte|cod_urf|qnt_estatistica|peso_liquido|valor_dolar|  urf_localizacao|          mercadoria|
+----+--------+--------------+------------+--------+------+---------------+-------+---------------+------------+-----------+-----------------+--------------------+
|2013|Setembro|      27101259|Metro cúbico|     249|    SP|       Marítima| 817800|           6449|     4959913|    4703835|  PORTO DE SANTOS|Outras gasolinas,...|
|2013|Novembro|      27101259|Metro cúbico|     611|    SP|       Marítima| 817800|           6501|     5243354|    4620039|  PORTO DE SANTOS|Outras gasolinas,...|
|2013|  Agosto|      27101259|Metro cúbico|     611|    SP|       Marítima| 817800|            221|      169964|     184589|  PORTO DE SANTOS|Outras gasolinas,...|
|2013|Setembro| 

In [ ]:
#mortar o dataframe no pyspark, usando o esquema
def montar_df(caminho, formato):
  df = ( spark.read.format(formato)
                 .option('header', 'true')
                 .option('inferschema', 'true')
                 .option('delimiter', ',')
                 .load(caminho))
  return df

In [ ]:
#chamar a função para criar a base com o esquema
# dfpais = montar_df(path_pais, 'csv')

In [ ]:
# dfip = montar_df(path_ip, 'parquet')

abrindo a base em spark

In [ ]:
df.show()

+--------+----------+-----------+--------+------+-------+---------------+------------+-----------+--------------------+----------------+---------------+----+
|     mes|mercadoria|cod_unidade|cod_pais|estado|cod_urf|qnt_estatistica|peso_liquido|valor_dolar|             produto|  unidade_medida|meio_transporte| ano|
+--------+----------+-----------+--------+------+-------+---------------+------------+-----------+--------------------+----------------+---------------+----+
|novembro|  73269090|         10|     493|    SP| 817700|           1409|        1409|      10031|Outras obras de f...|NUMERO (UNIDADE)|          AEREA|2021|
|   março|  73269090|         10|     249|    RJ| 817700|           2859|        2859|     459689|Outras obras de f...|NUMERO (UNIDADE)|          AEREA|2021|
|dezembro|  73269090|         10|     160|    RJ| 717700|            315|         315|      49945|Outras obras de f...|NUMERO (UNIDADE)|          AEREA|2021|
|dezembro|  73269090|         10|     275|    SP| 81

In [ ]:
df.describe().show()

Ordenações na base de exportação por produto

In [ ]:
#os 10 produtos exportados com maior valor em dolares no 10 anos.
dfexp_prod.orderBy(F.col('valor_dolar').desc()).show(10)

+----+--------+--------------+------------------+--------+------+---------------+-------+---------------+------------+-----------+------------------+--------------------+
| ano|     mes|cod_mercadoria|           unidade|cod_pais|estado|meio_transporte|cod_urf|qnt_estatistica|peso_liquido|valor_dolar|   urf_localizacao|          mercadoria|
+----+--------+--------------+------------------+--------+------+---------------+-------+---------------+------------+-----------+------------------+--------------------+
|2021|  Agosto|      26011100|Quilograma líquido|     160|    PA|       Marítima| 317903|    12873417000| 12873417000| 2102966398|      IRF SAO LUIS|Minérios de ferro...|
|2021|   Junho|      26011100|Quilograma líquido|     160|    PA|       Marítima| 317903|    11466550000| 11466550000| 1840229756|      IRF SAO LUIS|Minérios de ferro...|
|2021|Setembro|      26011100|Quilograma líquido|     160|    PA|       Marítima| 317903|    14300175000| 14300175000| 1808501568|      IRF SAO L

In [ ]:
#os 10 produtos exportados com maior volume no 10 anos.
dfexp_prod.orderBy(F.col('peso_liquido').desc()).show(10)

+----+--------+--------------+------------------+--------+------+---------------+-------+---------------+------------+-----------+--------------------+--------------------+
| ano|     mes|cod_mercadoria|           unidade|cod_pais|estado|meio_transporte|cod_urf|qnt_estatistica|peso_liquido|valor_dolar|     urf_localizacao|          mercadoria|
+----+--------+--------------+------------------+--------+------+---------------+-------+---------------+------------+-----------+--------------------+--------------------+
|2022|  Agosto|      89061000|    Número unidade|     827|    RJ|       Marítima| 717600|              1| 24030871000|    2013128|  PORTO DO RIO DE...|    Navios de guerra|
|2020|Setembro|      26011100|Quilograma líquido|     160|    PA|       Marítima| 317903|    17922689000| 17922689000| 1533003114|        IRF SAO LUIS|Minérios de ferro...|
|2019|   Julho|      26011100|Quilograma líquido|     160|    PA|       Marítima| 320151|    15465749000| 15465749000| 1193313692|  IRF

In [ ]:
#criar coluna com o valor das exportação por volume transportado
dfexp_prod = dfexp_prod.withColumn('dolar_por_vol', F.round(F.col('valor_dolar')/F.col('peso_liquido'), 2))

In [ ]:
#ver a coluna nova
dfexp_prod.show(10)

+----+---------+--------------+------------------+--------+------+---------------+-------+---------------+------------+-----------+-----------------+--------------------+-------------+
| ano|      mes|cod_mercadoria|           unidade|cod_pais|estado|meio_transporte|cod_urf|qnt_estatistica|peso_liquido|valor_dolar|  urf_localizacao|          mercadoria|dolar_por_vol|
+----+---------+--------------+------------------+--------+------+---------------+-------+---------------+------------+-----------+-----------------+--------------------+-------------+
|2013|    Março|      30067000|Quilograma líquido|     169|    SP|       Marítima| 817800|           1447|        1447|      27538|  PORTO DE SANTOS|Preparações em ge...|        19.03|
|2013|  Janeiro|      30067000|Quilograma líquido|     493|    SP|       Marítima| 817800|           1512|        1512|      26435|  PORTO DE SANTOS|Preparações em ge...|        17.48|
|2013|    Julho|      30067000|Quilograma líquido|     580|    SP|       Ma

In [ ]:
#os 10 produtos exportados com maior valor por volume.
dfexp_prod.orderBy(F.col('dolar_por_vol').desc()).show(10)

+----+--------+--------------+------------------+--------+------+--------------------+-------+---------------+------------+-----------+--------------------+--------------------+-------------+
| ano|     mes|cod_mercadoria|           unidade|cod_pais|estado|     meio_transporte|cod_urf|qnt_estatistica|peso_liquido|valor_dolar|     urf_localizacao|          mercadoria|dolar_por_vol|
+----+--------+--------------+------------------+--------+------+--------------------+-------+---------------+------------+-----------+--------------------+--------------------+-------------+
|2022|  Agosto|      27160000|Mil quilowatt hora|      63|    PR|Conduto rede de t...|1017500|         863702|           1|   85101443|     ALF  URUGUAIANA|    Energia elétrica|  8.5101443E7|
|2022|Setembro|      27160000|Mil quilowatt hora|      63|    PR|Conduto rede de t...|1017500|         695629|           1|   73193022|     ALF  URUGUAIANA|    Energia elétrica|  7.3193022E7|
|2022| Outubro|      27160000|Mil quilow

In [ ]:
#acrescentar um filtro, excluido, a mercadoria energia elétrico
dfexp_prod.filter(F.col('mercadoria')!='Energia elétrica').orderBy(F.col('dolar_por_vol').desc()).show(10)

+----+---------+--------------+------------------+--------+------+-----------------+-------+---------------+------------+-----------+--------------------+--------------------+-------------+
| ano|      mes|cod_mercadoria|           unidade|cod_pais|estado|  meio_transporte|cod_urf|qnt_estatistica|peso_liquido|valor_dolar|     urf_localizacao|          mercadoria|dolar_por_vol|
+----+---------+--------------+------------------+--------+------+-----------------+-------+---------------+------------+-----------+--------------------+--------------------+-------------+
|2018| Setembro|      97011000|    Número unidade|     249|    RJ|            Aérea| 717700|              1|           3|   25000000|  AEROPORTO INTER...|Quadros, pinturas...|   8333333.33|
|2018| Novembro|      71031000|Quilograma líquido|     767|    RJ|          Em mãos| 817600|              1|           1|    6355500|  AEROPORTO INTER...|Pedras preciosas ...|    6355500.0|
|2018| Setembro|      88023029|    Número unidade|

#Disponibilização das bases originais e tratadas para a bucket exclusiva do cliente EPOC

Instalação e configuração

In [ ]:
#instalar biblioteca da google cloud storage file system
!pip install gcsfs

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
#importar o pandas
import pandas as pd

In [ ]:
#importar o cloud storage
from google.cloud import storage
import os
from google.cloud.storage.blob import Blob 

Acesso ao cloud storage

In [ ]:
#Configura o acesso ao Cloud Storage pela chave da conta de serviço (biblioteca os)
serviceAccount = '/content/chave_bucket_JSON.json' 
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = serviceAccount

criação da bucket do cliente

In [ ]:
#da biblioteca storage.cliente usar a função para cria uma nova bucket
client = storage.Client()
bucket = client.create_bucket('coexp-epoc')
# bucket = storage.Client().create_bucket('coexp-epoc')
print(bucket)

In [ ]:
#verificar a existencia da bucket
try:
  bucket = client.get_bucket('coexp-epoc')
  print('bucket encontrada')
except:
  print('Sorry, essa bucket não existe')

bucket encontrada


In [ ]:
#listar as bucket
for bucket in client.list_buckets():
  print(bucket)

<Bucket: bc26-datasets-comex>
<Bucket: coexp-epoc>
<Bucket: dataflow-staging-us-central1-53174594931>
<Bucket: dataflow-staging-us-west1-53174594931>
<Bucket: dataproc-temp-sa-east1-53174594931-3xsvzgfc>
<Bucket: dataproc-temp-us-west1-53174594931-owoimmug>


#Acesso a bucket de trabalho

In [ ]:
#entra na bucket de trabalho
bucket_trab = client.get_bucket('bc26-datasets-comex')

In [ ]:
#listar os blob dentro de uma bucket
lista_blob = bucket_trab.list_blobs()
for blob in lista_blob:
  print(blob)

<Blob: bc26-datasets-comex, BQ_pipeline/, 1673026179326760>
<Blob: bc26-datasets-comex, BQ_pipeline/BQ_pipeline.csv, 1673028081537644>
<Blob: bc26-datasets-comex, BQ_pipeline/models/_exp_ncm, 1673283474389792>
<Blob: bc26-datasets-comex, BQ_pipeline/models/md_exp_mun, 1673387176001219>
<Blob: bc26-datasets-comex, BQ_pipeline/models/modelo_batch, 1673269324838074>
<Blob: bc26-datasets-comex, BQ_pipeline/staging/beamapp-root-0106181254-660754-in4y1ty0.1673028774.661024/apache_beam-2.43.0-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl, 1673028796531287>
<Blob: bc26-datasets-comex, BQ_pipeline/staging/beamapp-root-0106181254-660754-in4y1ty0.1673028774.661024/dataflow_python_sdk.tar, 1673028777601702>
<Blob: bc26-datasets-comex, BQ_pipeline/staging/beamapp-root-0106181254-660754-in4y1ty0.1673028774.661024/pipeline.pb, 1673028796899006>
<Blob: bc26-datasets-comex, BQ_pipeline/staging/beamapp-root-0106181502-265073-opaybmzv.1673028902.265298/apache_beam-2.43.0-cp38-cp38-manylinux_2_

#Acesso a bucket do cliente EPOC

In [ ]:
#acessar a bucket do cliente
bucket_epoc = client.get_bucket('coexp-epoc')

In [ ]:
#listar o conteudo da bucket do epoc

#criar varível para receber a informação
conteudo_bucket = bucket_epoc.list_blobs()

#as função list_blobs cria um iterado de http
print(conteudo_bucket)


In [ ]:
#para ver os 'blobs'/objetos na buscket, faça uma iteração
for blob in conteudo_bucket:
  print(blob)

<Blob: coexp-epoc, normalizada/classificacao_produtos, 1673891480520917>
<Blob: coexp-epoc, normalizada/combustivel_preco/produto=ETANOL HIDRATADO/2348148f42fa48c99a1cf494178b7b3d-0.parquet, 1673893828582527>
<Blob: coexp-epoc, normalizada/combustivel_preco/produto=GASOLINA ADITIVADA/2348148f42fa48c99a1cf494178b7b3d-0.parquet, 1673893828166838>
<Blob: coexp-epoc, normalizada/combustivel_preco/produto=GASOLINA COMUM/2348148f42fa48c99a1cf494178b7b3d-0.parquet, 1673893828605356>
<Blob: coexp-epoc, normalizada/combustivel_preco/produto=GLP/2348148f42fa48c99a1cf494178b7b3d-0.parquet, 1673893828699833>
<Blob: coexp-epoc, normalizada/combustivel_preco/produto=GNV/2348148f42fa48c99a1cf494178b7b3d-0.parquet, 1673893828274424>
<Blob: coexp-epoc, normalizada/combustivel_preco/produto=OLEO DIESEL S10/2348148f42fa48c99a1cf494178b7b3d-0.parquet, 1673893828595815>
<Blob: coexp-epoc, normalizada/combustivel_preco/produto=OLEO DIESEL/2348148f42fa48c99a1cf494178b7b3d-0.parquet, 1673893828690950>
<Blob: 

#Pegar um arquivo de uma bucket e enviar para outra

envio dos dataset original no escopo de 10 anos

In [ ]:
#listas do path das bases originais - dentro da bucket de trabalho, com apenas 10 anos nos formatos cvs ou parquet
path_originais = []
path = "sim"
while path !='fim':
  print('Digite fim para terminar')
  path = input("Digite o gsutil da base: ").lower()
  path_originais.append(path)
  

In [ ]:
len(path_originais)

5

In [ ]:
for i in range(len(path_originais)):
  if path_originais[i][-3:] == 'csv':
    print(path_originais[i])


vida.csv
sera.csv


In [ ]:
for i in range(len(path_originais)):
  print(path_originais[i], path_originais[i][-3:])  

amor mor
vida.csv csv
nada ada
sera.csv csv
fim fim


Storage Client

In [ ]:
#listas do path das bases tratadas - dentro da bucket de trabalho, com apenas 10 anos nos formatos cvs ou parquet
path_tratado = []
path = "sim"
while path !='fim':
  print('Digite fim para terminar')
  path = input("Digite o gsutil da base: ")
  path_originais.append(path)


In [ ]:
# ACESSO DA BUCKET
client = storage.Client()

# VARIÁVEL PARA RECEBER O NOME DA BUCKET 
bucket = client.get_bucket('bc26-datasets-comex')

# SELEÇÃO DO ARQUIVO NA BUCKET
bucket.blob('IMP_COMPLETA (NCM).csv')

# VARIÁVEL PARA RECEBER O CAMINHO DO ARQUIVO
path = 'gs://bc26-datasets-comex/IMP_COMPLETA (NCM).csv'

In [ ]:
#CRIAÇÃO DO DF A PARTIR DO CSV (BASE DE DADOS DE IMPORTAÇÃO, BASEADA NOS PRODUTOS)
df1 = pd.read_csv(path, sep=";", encoding='ISO-8859-1')

In [ ]:
# criação do caminho dentro da bucket (que ja está acessada) para salvar o parquet
path = 'gs://bc26-datasets-comex/IMP_2012-2022 (NCM).parquet'

In [ ]:
# SALVAMENTO DO ARQUIVO NO FORMATO PARQUET DENTRO DA BUCKET
df1.to_parquet(path, index=False, partition_cols=["CO_ANO"])

In [ ]:

bucket.blob('bc26-datasets-comex/EXP_COMPLETA (NCM).csv')

path = 'gs://bc26-datasets-comex/EXP_COMPLETA (NCM).csv'
df1 = pd.read_csv(path, sep=";")

#Enviar os dataset para o bucket do clinte EPOC

Links/caminhos para os dataset originais e normalizados

In [ ]:
#datasets originais

#em parquet - ok
exp_mun_orig_parquet = 'gs://bc26-datasets-comex/comex-original-parquet/EXP_COMPLETA_(MUN).parquet'
exp_ncm_orig_parquet = 'gs://bc26-datasets-comex/comex-original-parquet/EXP_COMPLETA_(NCM).parquet'
imp_mun_orig_parquet = 'gs://bc26-datasets-comex/IMP_2012-2022 (MUN).parquet'
imp_ncm_orig_parquet = 'gs://bc26-datasets-comex/IMP_2012-2022 (NCM).parquet'

#base com várias sheet
#aux_orig_excel ='gs://bc26-datasets-comex/TABELAS_AUXILIARES.xlsx'

#em xlsx - ok, salvo em csv
comb_preco_orig_2013_17 = 'gs://bc26-datasets-comex/combustível_semanal_xlsx/semanal-municipios-2013-a-2017.xlsx'
comb_preco_orig_2018_21 = 'gs://bc26-datasets-comex/combustível_semanal_xlsx/semanal-municipios-2018-a-2021.xlsx'
comb_preco_orig_2022 = 'gs://bc26-datasets-comex/combustível_semanal_xlsx/semanal-municipios-2022.xlsx'

#em csv
comb_qtd_orig_1 = 'gs://bc26-datasets-comex/comex-original-link/vendas-combustiveis-segmento-m3-2012-2022'
comb_qtd_orig_2 = 'gs://bc26-datasets-comex/comex-original-link/vendas-glp-tipo-vasilhame-m3-2007-2022'
comb_qtd_orig_3 = 'gs://bc26-datasets-comex/comex-original-link/vendas-oleo-diesel-tipo-m3-2013-2022'


#datasets normalizados
#em parquet ok
exp_mun_norm_parquet = 'gs://bc26-datasets-comex/comex-tratado/EXP_2013-2022_tratado(MUN).parquet'
exp_ncm_norm_parquet = 'gs://bc26-datasets-comex/comex-tratado/EXP_2013-2022_tratado(NCM).parquet'
imp_mun_norm_parquet = 'gs://bc26-datasets-comex/comex-tratado/IMP_2012-2022_tratada (MUN).parquet'
imp_ncm_norm_parquet = 'gs://bc26-datasets-comex/comex-tratado/IMP_2012-2022_tratada (NCM).parquet'
comb_preco_norm_parquet = 'gs://bc26-datasets-comex/preco-combustiveis-2013-2022-TRATAMENTO-OK'
comb_qdt_norm_parquet = 'gs://bc26-datasets-comex/combustivel-tratado/volume_preco_combustiveis_mensal_2013_2022'

#auxiliar em csv - ok
pais = 'gs://bc26-datasets-comex/comex-tratado/nome-pais'
classes = 'gs://bc26-datasets-comex/comex-tratado/imp-exp-classificacoes'



Enviando os dataset em csv - dataset auxiliares

In [ ]:
#criar um nome para um nome e caminho para o dataset
#bucket_epoc.blob('/pasta/nome que quero')
bucket_epoc.blob('normalizada/classificacao_produtos')

<Blob: coexp-epoc, normalizada/classificacao_produtos, None>

In [ ]:
#ler o data set no bucket de trabalho
df_csv = pd.read_csv(classes)
#classes é o caminho do arquivo na buskbet de trabalho

In [ ]:
#conferir o dataset
df_csv.head(3)

,ncm,produto,sh6,sh6_mercadoria,sh6_posicao,sh6_capitulo,sh6_secao,cuci,cuci_item,cuci_subgrupo,...,cgce,cgce_economica_n3,cgce_n2,cgce_n1,isic,isic_class_industrial,isic_grupo,isic_divisao,isic_secao,siit_setor_industrial
0,1011010,Cavalos reprodutores de raça pura,10110,"Animais vivos das espécies cavalar, asinina e ...","Cavalos, asininos e muares, vivos",Animais vivos,Animais vivos e produtos do reino animal,0015,"Cavalos, asnos, mulas e burros vivos","Cavalos, asnos, mulas e burros vivos",...,110,Bens de capital (exceto equipamentos de transp...,"Bens de capital, exceto equipamentos de transp...",BENS DE CAPITAL (BK),142,Criação de cavalos e outros equinos,Produção animal,"Produção vegetal, animal e caça",Agropecuária,PRODUTOS N.C.I.T
1,1011090,"Animais da espécie asinina/muar, reprodutores,...",10110,"Animais vivos das espécies cavalar, asinina e ...","Cavalos, asininos e muares, vivos",Animais vivos,Animais vivos e produtos do reino animal,0015,"Cavalos, asnos, mulas e burros vivos","Cavalos, asnos, mulas e burros vivos",...,110,Bens de capital (exceto equipamentos de transp...,"Bens de capital, exceto equipamentos de transp...",BENS DE CAPITAL (BK),142,Criação de cavalos e outros equinos,Produção animal,"Produção vegetal, animal e caça",Agropecuária,PRODUTOS N.C.I.T
2,1011100,"Cavalos reprodutores, de raça pura",10111,"Cavalos reprodutores, de raça pura","Cavalos, asininos e muares, vivos",Animais vivos,Animais vivos e produtos do reino animal,0015,"Cavalos, asnos, mulas e burros vivos","Cavalos, asnos, mulas e burros vivos",...,110,Bens de capital (exceto equipamentos de transp...,"Bens de capital, exceto equipamentos de transp...",BENS DE CAPITAL (BK),142,Criação de cavalos e outros equinos,Produção animal,"Produção vegetal, animal e caça",Agropecuária,PRODUTOS N.C.I.T


In [ ]:
#criar um caminho para esse novo arquivo
path = 'gs://coexp-epoc/normalizada/classificacao_produtos'
#salvar o arquivo na bucket do cliente EPOC
df_csv.to_csv(path, index=False)


In [ ]:
#criar um nome para um nome e caminho para o dataset
bucket_epoc.blob('normalizada/nome_pais')

<Blob: coexp-epoc, normalizada/nome_pais, None>

In [ ]:
#ler o data set no bucket de trabalho
df_csv = pd.read_csv(pais)
#pais é o caminho do arquivo na buskbet de trabalho

In [ ]:
#conferir o dataset
df_csv.head(3)

,cod_pais,pais
0,0,Não Definido
1,13,Afeganistão
2,15,"Aland, Ilhas"


In [ ]:
#criar um caminho para esse novo arquivo
path = 'gs://coexp-epoc/normalizada/nome_paises'
#salvar o arquivo na bucket do cliente EPOC
df_csv.to_csv(path, index=False)

Enviar os dataset em parquet - dataset normalizados

In [ ]:
#criar um nome para um nome e caminho para o dataset
bucket_epoc.blob('normalizada/exportacao_municipio')
#ler o data set no bucket de trabalho
df = pd.read_parquet(exp_mun_norm_parquet)
#conferir o dataset
df.head(3)

,mes,SH4,cod_pais,estado,peso_liquido,valor_dolar,municipio,ano
0,Junho,8501,63,PR,780,76547,Curitiba,2013
1,Junho,8448,361,PR,1,110,Curitiba,2013
2,Março,8543,589,PR,19,3690,Curitiba,2013


In [ ]:
#criar um caminho para esse novo arquivo
path = 'gs://coexp-epoc/normalizada/exportacao_municipio'
#salvar o arquivo na bucket do cliente EPOC
df.to_parquet(path, index=False, partition_cols=['ano'])

In [ ]:
#criar um nome para um nome e caminho para o dataset
bucket_epoc.blob('normalizada/exportacao_produto')
#ler o data set no bucket de trabalho
df = pd.read_parquet(exp_ncm_norm_parquet )
#conferir o dataset
df.head(3)

,mes,cod_mercadoria,unidade,cod_pais,estado,meio_transporte,cod_urf,qnt_estatistica,peso_liquido,valor_dolar,urf_localizacao,mercadoria,ano
0,Abril,84833029,Número unidade,676,RS,Aérea,1017600,10,2,123,AEROPORTO SALGADO FILHO PORTO ALEGRE,Outros bronzes,2013
1,Abril,84833029,Número unidade,845,RS,Aérea,1017600,1,10,43,AEROPORTO SALGADO FILHO PORTO ALEGRE,Outros bronzes,2013
2,Fevereiro,84833029,Número unidade,196,RS,Aérea,1017600,1,0,19,AEROPORTO SALGADO FILHO PORTO ALEGRE,Outros bronzes,2013


In [ ]:
#criar um caminho para esse novo arquivo
path = 'gs://coexp-epoc/normalizada/exportacao_produto'
#salvar o arquivo na bucket do cliente EPOC
df.to_parquet(path, index=False, partition_cols=['ano'])

In [ ]:
#criar um nome para um nome e caminho para o dataset
bucket_epoc.blob('normalizada/importacao_municipio')
#ler o data set no bucket de trabalho
df = pd.read_parquet(imp_mun_norm_parquet)
#conferir o dataset
df.head(3)

,mes,SH4,cod_pais,estado,cod_municipio,peso_liquido,valor_dolar,descricoes__sistema_harmonizado,municipio,ano
0,janeiro,6115,160,RS,4314902,2512,37327,"Polímeros de etileno, em formas primárias",Porto Alegre,2012
1,fevereiro,8481,23,RS,4314902,80,8444,"Fatos de saia-casaco, conjuntos, casacos, vest...",Porto Alegre,2012
2,dezembro,7009,160,RS,4314902,23700,20412,"Obras de carpintaria para construções, incluíd...",Porto Alegre,2012


In [ ]:
#criar um caminho para esse novo arquivo
path = 'gs://coexp-epoc/normalizada/importacao_municipio'
#salvar o arquivo na bucket do cliente EPOC
df.to_parquet(path, index=False, partition_cols=['ano'])

In [ ]:
#criar um nome para um nome e caminho para o dataset
bucket_epoc.blob('normalizada/importacao_produto')
#ler o data set no bucket de trabalho
df = pd.read_parquet(imp_ncm_norm_parquet)
#conferir o dataset
df.head(3)

,mes,mercadoria,cod_unidade,cod_pais,estado,cod_urf,qnt_estatistica,peso_liquido,valor_dolar,produto,unidade_medida,meio_transporte,ano
0,março,39095021,10,190,CE,317700,211,211,3600,"Poliuretanos hidroxilados, com propriedades ad...",NUMERO (UNIDADE),AEREA,2012
1,junho,39095021,10,245,RS,1017600,50,50,459,"Poliuretanos hidroxilados, com propriedades ad...",NUMERO (UNIDADE),AEREA,2012
2,março,39095021,10,23,RS,1017600,1600,1600,21300,"Poliuretanos hidroxilados, com propriedades ad...",NUMERO (UNIDADE),AEREA,2012


In [ ]:
#criar um caminho para esse novo arquivo
path = 'gs://coexp-epoc/normalizada/importacao_produto'
#salvar o arquivo na bucket do cliente EPOC
df.to_parquet(path, index=False, partition_cols=['ano'])

In [ ]:
#criar um nome para um nome e caminho para o dataset
bucket_epoc.blob('normalizada/combustivel_preco')
#ler o data set no bucket de trabalho
df = pd.read_parquet(comb_preco_norm_parquet)
#conferir o dataset
df.head(3)

,data_inicial,data_final,regiao,estado,municipio,produto,num_postos_pesquisados,unidade_medida,preco_medio_revenda,desvio_padrao_revenda,preco_min_revenda,preco_max_revenda,margem_media_revenda,coef_variacao_revenda
0,2012-12-30,2013-01-05,CENTRO OESTE,DISTRITO FEDERAL,BRASILIA,ETANOL HIDRATADO,80,R$/l,2.273,0.023,2.19,2.299,0.405,0.010
1,2012-12-30,2013-01-05,CENTRO OESTE,GOIAS,AGUAS LINDAS DE GOIAS,ETANOL HIDRATADO,5,R$/l,2.134,0.043,2.09,2.190,0.268,0.020
2,2012-12-30,2013-01-05,CENTRO OESTE,GOIAS,ANAPOLIS,ETANOL HIDRATADO,16,R$/l,1.865,0.055,1.69,1.899,0.360,0.029


In [ ]:
#criar um caminho para esse novo arquivo
path = 'gs://coexp-epoc/normalizada/combustivel_preco'
#salvar o arquivo na bucket do cliente EPOC
df.to_parquet(path, index=False, partition_cols=['produto'])

In [ ]:
#criar um nome para um nome e caminho para o dataset
bucket_epoc.blob('normalizada/combustivel_quantidade')
#ler o data set no bucket de trabalho
df = pd.read_parquet(comb_qdt_norm_parquet)
#conferir o dataset
df.head(3)

,mes,estado,produto,qtde_vendas,num_postos_pesquisados,preco_medio_revenda,desvio_padrao_revenda,preco_min_revenda,preco_max_revenda,margem_media_revenda,coef_variacao_revenda,ano
0,SETEMBRO,AMAZONAS,GLP,1999.79,30.0,45.27,0.658,44.08,46.33,9.239,0.015,2013
1,MARÇO,AMAZONAS,OLEO DIESEL S10,3391000.00,5.0,2.58,0.013,2.56,2.60,0.394,0.005,2013
2,FEVEREIRO,BAHIA,ETANOL HIDRATADO,22144450.00,12.0,2.23,0.081,2.11,2.35,0.308,0.036,2013


In [ ]:
#criar um caminho para esse novo arquivo
path = 'gs://coexp-epoc/normalizada/combustivel_quantidade'
#salvar o arquivo na bucket do cliente EPOC
df.to_parquet(path, index=False, partition_cols=['produto'])

Enviar os dataset em parquet - dataset original

In [ ]:
#criar um nome para um nome e caminho para o dataset
bucket_epoc.blob('original/exportacao_municipio')
#ler o data set no bucket de trabalho
df = pd.read_parquet(exp_mun_orig_parquet)
#conferir o dataset
df.head(3)

,CO_MES,SH4,CO_PAIS,SG_UF_MUN,CO_MUN,KG_LIQUIDO,VL_FOB,CO_ANO
0,4,4409,249,PA,1501808,39579,47495,2012
1,4,7415,63,SP,3452205,3,53,2012
2,3,8504,239,SP,3454102,4760,88750,2012


In [ ]:
#criar um caminho para esse novo arquivo
path = 'gs://coexp-epoc/original/exportacao_municipio'
#salvar o arquivo na bucket do cliente EPOC
df.to_parquet(path, index=False, partition_cols=['CO_ANO'])

In [ ]:
#criar um nome para um nome e caminho para o dataset
bucket_epoc.blob('original/exportacao_produto')
#ler o data set no bucket de trabalho
df = pd.read_parquet(exp_ncm_orig_parquet)
#conferir o dataset
df.head(3)

,CO_MES,CO_NCM,CO_UNID,CO_PAIS,SG_UF_NCM,CO_VIA,CO_URF,QT_ESTAT,KG_LIQUIDO,VL_FOB,CO_ANO
0,6,87089100,11,97,PR,7,145200,90,1194,21925,2012
1,6,33051000,10,249,SP,1,817800,3763,3763,17164,2012
2,6,94035000,11,628,RS,1,1017700,19576,561623,1490086,2012


In [ ]:
#criar um caminho para esse novo arquivo
path = 'gs://coexp-epoc/original/exportacao_produto'
#salvar o arquivo na bucket do cliente EPOC
df.to_parquet(path, index=False, partition_cols=['CO_ANO'])

In [ ]:
#criar um nome para um nome e caminho para o dataset
bucket_epoc.blob('original/importacao_municipio')
#ler o data set no bucket de trabalho
df = pd.read_parquet(imp_mun_orig_parquet)
#conferir o dataset
df.head(3)

,CO_MES,SH4,CO_PAIS,SG_UF_MUN,CO_MUN,KG_LIQUIDO,VL_FOB,CO_ANO
0,12,8414,361,SP,3448906,129,14934,2012
1,11,3819,249,SP,3425904,5,211,2012
2,8,6201,160,MG,3136702,6,107,2012


In [ ]:
#criar um caminho para esse novo arquivo
path = 'gs://coexp-epoc/original/importacao_municipio'
#salvar o arquivo na bucket do cliente EPOC
df.to_parquet(path, index=False, partition_cols=['CO_ANO'])

In [ ]:
#criar um nome para um nome e caminho para o dataset
bucket_epoc.blob('original/importacao_produto')
#ler o data set no bucket de trabalho
df = pd.read_parquet(imp_ncm_orig_parquet)
#conferir o dataset
df.head(3)

,CO_MES,CO_NCM,CO_UNID,CO_PAIS,SG_UF_NCM,CO_VIA,CO_URF,QT_ESTAT,KG_LIQUIDO,VL_FOB,VL_FRETE,VL_SEGURO,CO_ANO
0,10,85441990,10,399,SP,4,817700,0,0,134,24,0,2012
1,10,85115010,11,149,SP,4,817600,1,1,36833,21,11,2012
2,3,90013000,11,365,SP,4,817600,6950,153,20755,992,8,2012


In [ ]:
#criar um caminho para esse novo arquivo
path = 'gs://coexp-epoc/original/importacao_produto'
#salvar o arquivo na bucket do cliente EPOC
df.to_parquet(path, index=False, partition_cols=['CO_ANO'])

Enviar os dataset em xlsx - dataset original

In [ ]:
#criar um nome para um nome e caminho para o dataset
bucket_epoc.blob('original/combustivel_preco/combusitvel_municipios_2013_2017')
#ler o data set no bucket de trabalho
df = pd.read_excel(comb_preco_orig_2013_17, skiprows=14)
#conferir o dataset
df.head(3)

,DATA INICIAL,DATA FINAL,REGIÃO,ESTADO,MUNICÍPIO,PRODUTO,NÚMERO DE POSTOS PESQUISADOS,UNIDADE DE MEDIDA,PREÇO MÉDIO REVENDA,DESVIO PADRÃO REVENDA,PREÇO MÍNIMO REVENDA,PREÇO MÁXIMO REVENDA,MARGEM MÉDIA REVENDA,COEF DE VARIAÇÃO REVENDA,PREÇO MÉDIO DISTRIBUIÇÃO,DESVIO PADRÃO DISTRIBUIÇÃO,PREÇO MÍNIMO DISTRIBUIÇÃO,PREÇO MÁXIMO DISTRIBUIÇÃO,COEF DE VARIAÇÃO DISTRIBUIÇÃO
0,2012-12-30,2013-01-05,CENTRO OESTE,DISTRITO FEDERAL,BRASILIA,ETANOL HIDRATADO,80,R$/l,2.273,0.022864,2.19,2.299,0.405,0.010,1.868,0.016954,1.8429,1.8873,0.009
1,2012-12-30,2013-01-05,CENTRO OESTE,GOIAS,AGUAS LINDAS DE GOIAS,ETANOL HIDRATADO,5,R$/l,2.134,0.043359,2.09,2.190,0.268,0.020,1.866,0.020789,1.8512,1.8806,0.011
2,2012-12-30,2013-01-05,CENTRO OESTE,GOIAS,ANAPOLIS,ETANOL HIDRATADO,16,R$/l,1.865,0.054575,1.69,1.899,0.36,0.029,1.505,0.044552,1.4599,1.5527,0.03


In [ ]:
#criar um caminho para esse novo arquivo
path = 'gs://coexp-epoc/original/combustivel_preco/combusitvel_municipios_2013_2017'
#salvar o arquivo na bucket do cliente EPOC
df.to_csv(path, index=False)

In [ ]:
#criar um nome para um nome e caminho para o dataset
bucket_epoc.blob('original/combustivel_preco/combusitvel_municipios_2018_2021')
#ler o data set no bucket de trabalho
df = pd.read_excel(comb_preco_orig_2018_21, skiprows=13)
#conferir o dataset
df.head(3)

,DATA INICIAL,DATA FINAL,REGIÃO,ESTADO,MUNICÍPIO,PRODUTO,NÚMERO DE POSTOS PESQUISADOS,UNIDADE DE MEDIDA,PREÇO MÉDIO REVENDA,DESVIO PADRÃO REVENDA,PREÇO MÍNIMO REVENDA,PREÇO MÁXIMO REVENDA,MARGEM MÉDIA REVENDA,COEF DE VARIAÇÃO REVENDA,PREÇO MÉDIO DISTRIBUIÇÃO,DESVIO PADRÃO DISTRIBUIÇÃO,PREÇO MÍNIMO DISTRIBUIÇÃO,PREÇO MÁXIMO DISTRIBUIÇÃO,COEF DE VARIAÇÃO DISTRIBUIÇÃO
0,2017-12-31,2018-01-06,NORTE,PARA,ABAETETUBA,ETANOL HIDRATADO,2,R$/l,3.920,0.254558,3.740,4.100,-,0.065,-,-,-,-,-
1,2017-12-31,2018-01-06,NORDESTE,MARANHAO,ACAILANDIA,ETANOL HIDRATADO,2,R$/l,3.560,0.027577,3.540,3.579,-,0.008,-,-,-,-,-
2,2017-12-31,2018-01-06,SUDESTE,SAO PAULO,ADAMANTINA,ETANOL HIDRATADO,8,R$/l,2.526,0.047730,2.449,2.590,0.172,0.019,2.354,0.014126,2.3413,2.3692,0.006


In [ ]:
#criar um caminho para esse novo arquivo
path = 'gs://coexp-epoc/original/combustivel_preco/combusitvel_municipios_2018_2021'
#salvar o arquivo na bucket do cliente EPOC
df.to_csv(path, index=False)

In [ ]:
#criar um nome para um nome e caminho para o dataset
bucket_epoc.blob('original/combustivel_preco/combusitvel_municipios_2022')
#ler o data set no bucket de trabalho
df = pd.read_excel(comb_preco_orig_2022, skiprows=11)
#conferir o dataset
df.head(3)

,DATA INICIAL,DATA FINAL,REGIÃO,ESTADO,MUNICÍPIO,PRODUTO,NÚMERO DE POSTOS PESQUISADOS,UNIDADE DE MEDIDA,PREÇO MÉDIO REVENDA,DESVIO PADRÃO REVENDA,PREÇO MÍNIMO REVENDA,PREÇO MÁXIMO REVENDA,MARGEM MÉDIA REVENDA,COEF DE VARIAÇÃO REVENDA
0,2022-01-02,2022-01-08,SUDESTE,SAO PAULO,ADAMANTINA,ETANOL HIDRATADO,7,R$/l,4.656,0.077786,4.580,4.790,-,0.017
1,2022-01-02,2022-01-08,CENTRO OESTE,GOIAS,AGUAS LINDAS DE GOIAS,ETANOL HIDRATADO,4,R$/l,5.199,0.000500,5.198,5.199,-,0.000
2,2022-01-02,2022-01-08,SUL,RIO GRANDE DO SUL,ALEGRETE,ETANOL HIDRATADO,2,R$/l,6.845,0.642760,6.390,7.299,-,0.094


In [ ]:
#criar um caminho para esse novo arquivo
path = 'gs://coexp-epoc/original/combustivel_preco/combusitvel_municipios_2022'
#salvar o arquivo na bucket do cliente EPOC
df.to_csv(path, index=False)

Enviar os dataset em csv - dataset original

In [ ]:
#criar um nome para um nome e caminho para o dataset
bucket_epoc.blob('original/combustivel_quantidade/combusitvel_vendas_segmento_m3_2012_2022')
#ler o data set no bucket de trabalho
df = pd.read_csv(comb_qtd_orig_1, sep=';')
#conferir o dataset
df.head(3)

,ANO,MÊS,UNIDADE DA FEDERAÇÃO,PRODUTO,SEGMENTO,VENDAS
0,2012,JAN,RONDÔNIA,ETANOL HIDRATADO,TRR,0
1,2012,DEZ,DISTRITO FEDERAL,ETANOL HIDRATADO,CONSUMIDOR FINAL,35
2,2012,NOV,DISTRITO FEDERAL,ETANOL HIDRATADO,CONSUMIDOR FINAL,28


In [ ]:
#criar um caminho para esse novo arquivo
path = 'gs://coexp-epoc/original/combustivel_quantidade/combusitvel_vendas_segmento_m3_2012_2022'
#salvar o arquivo na bucket do cliente EPOC
df.to_csv(path, index=False)

In [ ]:
#criar um nome para um nome e caminho para o dataset
bucket_epoc.blob('original/combustivel_quantidade/combusitvel_vendas_glp_m3_2007_2022')
#ler o data set no bucket de trabalho
df = pd.read_csv(comb_qtd_orig_2, sep=';')
#conferir o dataset
df.head(3)

,ANO,MÊS,GRANDE REGIÃO,UNIDADE DA FEDERAÇÃO,VASILHAME,VENDAS
0,2007,JAN,REGIÃO NORTE,RONDÔNIA,GLP - Até P13,"5296,306159"
1,2007,FEV,REGIÃO NORTE,RONDÔNIA,GLP - Até P13,"5208,173913"
2,2007,MAR,REGIÃO NORTE,RONDÔNIA,GLP - Até P13,"5716,313406"


In [ ]:
#criar um caminho para esse novo arquivo
path = 'gs://coexp-epoc/original/combustivel_quantidade/combusitvel_vendas_glp_m3_2007_2022'
#salvar o arquivo na bucket do cliente EPOC
df.to_csv(path, index=False)

In [ ]:
#criar um nome para um nome e caminho para o dataset
bucket_epoc.blob('original/combustivel_quantidade/combusitvel_vendas_oleo_diesel_m3_2013_2022')
#ler o data set no bucket de trabalho
df = pd.read_csv(comb_qtd_orig_3, sep=';')
#conferir o dataset
df.head(3)

,ANO,MÊS,GRANDE REGIÃO,UNIDADE DA FEDERAÇÃO,PRODUTO,VENDAS
0,2013,JAN,REGIÃO NORTE,RONDÔNIA,ÓLEO DIESEL S-10,"3517,6"
1,2013,FEV,REGIÃO NORTE,RONDÔNIA,ÓLEO DIESEL S-10,"3681,7"
2,2013,MAR,REGIÃO NORTE,RONDÔNIA,ÓLEO DIESEL S-10,"4700,67"


In [ ]:
#criar um caminho para esse novo arquivo
path = 'gs://coexp-epoc/original/combustivel_quantidade/combusitvel_vendas_oleo_diesel_m3_2013_2022'
#salvar o arquivo na bucket do cliente EPOC
df.to_csv(path, index=False)